# Segmenting and Clustering Neighborhoods in Toronto

In [181]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

import requests

from geopy.geocoders import Nominatim

import folium

from pandas import json_normalize


In [182]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [183]:
data = requests.get(url).text
data

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"13be1a6b-7697-49b5-bef8-2507e240483d","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1019189119,"wgRevisionId":1019189119,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Wikipedia semi-protec

In [184]:
soup = BeautifulSoup(data, 'html5lib')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"13be1a6b-7697-49b5-bef8-2507e240483d","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1019189119,"wgRevisionId":1019189119,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Wikipedia

In [185]:
table = soup.find('table')
print(table.prettify())

<table cellpadding="2" cellspacing="0" rules="all" style="width:100%; border-collapse:collapse; border:1px solid #ccc;">
 <tbody>
  <tr>
   <td style="width:11%; vertical-align:top; color:#ccc;">
    <p>
     <b>
      M1A
     </b>
     <br/>
     <span style="font-size:85%;">
      <i>
       Not assigned
      </i>
     </span>
    </p>
   </td>
   <td style="width:11%; vertical-align:top; color:#ccc;">
    <p>
     <b>
      M2A
     </b>
     <br/>
     <span style="font-size:85%;">
      <i>
       Not assigned
      </i>
     </span>
    </p>
   </td>
   <td style="width:11%; vertical-align:top;">
    <p>
     <b>
      M3A
     </b>
     <br/>
     <span style="font-size:85%;">
      <a href="/wiki/North_York" title="North York">
       North York
      </a>
      <br/>
      (
      <a href="/wiki/Parkwoods" title="Parkwoods">
       Parkwoods
      </a>
      )
     </span>
    </p>
   </td>
   <td style="width:11%; vertical-align:top;">
    <p>
     <b>
      M4A
     </b>
 

In [186]:
table_contents = []
for row in table.findAll('td'):
    cell = {}
    if row.span.text == "Not assigned":
        pass
    else:
        print(row.span.text)
        print(row.p.text)
        print('\n')
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

North York(Parkwoods)
M3ANorth York(Parkwoods)



North York(Victoria Village)
M4ANorth York(Victoria Village)



Downtown Toronto(Regent Park / Harbourfront)
M5ADowntown Toronto(Regent Park / Harbourfront)



North York(Lawrence Manor / Lawrence Heights)
M6ANorth York(Lawrence Manor / Lawrence Heights)



Queen's Park(Ontario Provincial Government)
M7AQueen's Park(Ontario Provincial Government)



Etobicoke(Islington Avenue)
M9AEtobicoke(Islington Avenue)



Scarborough(Malvern / Rouge)
M1BScarborough(Malvern / Rouge)



North York(Don Mills)North
M3BNorth York(Don Mills)North



East York(Parkview Hill / Woodbine Gardens)
M4BEast York(Parkview Hill / Woodbine Gardens)



Downtown Toronto(Garden District, Ryerson)
M5BDowntown Toronto(Garden District, Ryerson)



North York(Glencairn)
M6BNorth York(Glencairn)



Etobicoke(West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale)
M9BEtobicoke(West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale)



In [187]:
toronto_data = pd.DataFrame(table_contents)

In [188]:
toronto_data

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [189]:
toronto_data.shape

(103, 3)

In [190]:
df = pd.read_csv('Geospatial_Coordinates.csv')
df = df.rename(columns={'Postal Code' : 'PostalCode'})
df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [191]:
toronto_df_coordinates = pd.merge(toronto_data, df, how='inner', left_on=['PostalCode'], right_on=['PostalCode'])
toronto_df_coordinates.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [192]:
toronto_df_coordinates.shape

(103, 5)

## Filter for Borough names containing Toronto

In [193]:
toronto_df_filtered = pd.DataFrame(toronto_df_coordinates[toronto_df_coordinates['Borough'].str.contains("Toronto")].reset_index(drop=True))

In [194]:
toronto_df_filtered.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
9,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106


In [195]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent='toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("Toronto coordinates are: {}, {}".format(latitude, longitude))

Toronto coordinates are: 43.6534817, -79.3839347


In [196]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start = 12)

for lat, lng, label in zip(toronto_df_filtered['Latitude'], toronto_df_filtered['Longitude'], toronto_df_filtered['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        fill=True,
        popup=label,
        color='green',
        fill_color='red',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto)

map_toronto

## Define Foursquare Credentials and Version

In [197]:
CLIENT_ID = 'PK5KQQ4KQ1K33DQLABOWPVY1VA0GRFD4LEAUXECZ4MOYJW1U' # your Foursquare ID
CLIENT_SECRET = 'ZEQHN4AL13WB0Y1C3SA3ESIVORWM4EPANN50PRK1AFTODBBU' # your Foursquare Secret
ACCESS_TOKEN = 'ERZERBXUIXCREQXLJJ5YJRMGHKX0GECSL5I5F4CSO0RGCIJP' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PK5KQQ4KQ1K33DQLABOWPVY1VA0GRFD4LEAUXECZ4MOYJW1U
CLIENT_SECRET:ZEQHN4AL13WB0Y1C3SA3ESIVORWM4EPANN50PRK1AFTODBBU


### Let's explore the one of the neighborhood in our dataframe.

### Explore The Beaches Neighborhood

In [198]:
toronto_df_filtered.loc[3, 'Neighborhood']

'The Beaches'

In [199]:
neighborhood_lat = toronto_df_filtered.loc[3, 'Latitude']
neighborhood_lng = toronto_df_filtered.loc[3, 'Longitude']
neighborhood_name = toronto_df_filtered.loc[3, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_lat, 
                                                               neighborhood_lng))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


### let's get the top 100 venues that are in "The Beaches" neighborhood within a radius of 500 meters.

In [200]:
radius = 500

url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&oauth_token={}&ll={},{}&v={}&radius={}&limit={}".format(CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, neighborhood_lat, neighborhood_lng, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=PK5KQQ4KQ1K33DQLABOWPVY1VA0GRFD4LEAUXECZ4MOYJW1U&client_secret=ZEQHN4AL13WB0Y1C3SA3ESIVORWM4EPANN50PRK1AFTODBBU&oauth_token=ERZERBXUIXCREQXLJJ5YJRMGHKX0GECSL5I5F4CSO0RGCIJP&ll=43.67635739999999,-79.2930312&v=20180604&radius=500&limit=100'

In [201]:
results = requests.get(url).json()
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues)
nearby_venues.head()

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,...,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.venuePage.id,venue.location.crossStreet
0,e-0-4ad4c062f964a52011f820e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4ad4c062f964a52011f820e3,The Big Carrot Natural Food Market,125 Southwood Dr,43.678879,-79.297734,"[{'label': 'display', 'lat': 43.678879, 'lng':...",471,...,CA,Toronto,ON,Canada,"[125 Southwood Dr, Toronto ON M4E 0B8, Canada]","[{'id': '50aa9e744b90af0d42d5de0e', 'name': 'H...",0,[],75150878,NaN
1,e-0-4b8daea1f964a520480833e3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4b8daea1f964a520480833e3,Grover Pub and Grub,676 Kingston Rd.,43.679181,-79.297215,"[{'label': 'display', 'lat': 43.67918143494101...",460,...,CA,Toronto,ON,Canada,"[676 Kingston Rd. (at Main St.), Toronto ON M4...","[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",0,[],NaN,at Main St.
2,e-0-4bd461bc77b29c74a07d9282-2,0,"[{'summary': 'This spot is popular', 'type': '...",4bd461bc77b29c74a07d9282,Glen Manor Ravine,Glen Manor,43.676821,-79.293942,"[{'label': 'display', 'lat': 43.67682094413784...",89,...,CA,Toronto,ON,Canada,"[Glen Manor (Queen St.), Toronto ON, Canada]","[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",0,[],NaN,Queen St.
3,e-0-4dbc8fe96a23e294ba3237bd-3,0,"[{'summary': 'This spot is popular', 'type': '...",4dbc8fe96a23e294ba3237bd,Glen Stewart Park,131 Glen Manor Drive,43.675278,-79.294647,"[{'label': 'display', 'lat': 43.67527822698259...",177,...,CA,Toronto,ON,Canada,"[131 Glen Manor Drive, Toronto ON, Canada]","[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[],NaN,NaN
4,e-0-50f9bbcc5d24acebc25935b5-4,0,"[{'summary': 'This spot is popular', 'type': '...",50f9bbcc5d24acebc25935b5,Domino's Pizza,670 Kingston Road,43.679058,-79.297382,"[{'label': 'display', 'lat': 43.67905757954377...",461,...,CA,Toronto,ON,Canada,"[670 Kingston Road, Toronto ON M4E 1R4, Canada]","[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",0,[],NaN,NaN


In [202]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
#     print(categories_list[0].keys())
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

In [203]:
filtered_cols = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_cols]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
1,Grover Pub and Grub,Pub,43.679181,-79.297215
2,Glen Manor Ravine,Trail,43.676821,-79.293942
3,Glen Stewart Park,Park,43.675278,-79.294647
4,Domino's Pizza,Pizza Place,43.679058,-79.297382


In [204]:
nearby_venues.columns = [col.split('.')[-1] for col in nearby_venues.columns]
print(nearby_venues.head())
print(nearby_venues.shape)

                                 name         categories        lat        lng
0  The Big Carrot Natural Food Market  Health Food Store  43.678879 -79.297734
1                 Grover Pub and Grub                Pub  43.679181 -79.297215
2                   Glen Manor Ravine              Trail  43.676821 -79.293942
3                   Glen Stewart Park               Park  43.675278 -79.294647
4                      Domino's Pizza        Pizza Place  43.679058 -79.297382
(9, 4)


## Explore Neighborhoods in all borough names having Toronto

In [205]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [206]:
toronto_borough_venues = getNearbyVenues(toronto_df_filtered['Neighborhood'],
                                   toronto_df_filtered['Latitude'],
                                   toronto_df_filtered['Longitude'])

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [207]:
toronto_borough_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [208]:
toronto_borough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,46,46,46,46,46,46
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",18,18,18,18,18,18
Central Bay Street,61,61,61,61,61,61
Christie,16,16,16,16,16,16
Church and Wellesley,66,66,66,66,66,66
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,29,29,29,29,29,29
Davisville North,10,10,10,10,10,10


In [209]:
print("There are {} unique categories: ".format(len(toronto_borough_venues['Venue Category'].unique())))

There are 215 unique categories: 


### Analyze Each Neighborhood

In [210]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_borough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_borough_venues['Neighborhood'] 

toronto_onehot.head()

,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [211]:
neighborhood_col = pd.DataFrame(toronto_onehot['Neighborhood'])
neighborhood_col

,Neighborhood
0,"Regent Park, Harbourfront"
1,"Regent Park, Harbourfront"
2,"Regent Park, Harbourfront"
3,"Regent Park, Harbourfront"
4,"Regent Park, Harbourfront"
...,...
1490,Enclave of M4L
1491,Enclave of M4L
1492,Enclave of M4L
1493,Enclave of M4L


In [212]:
other_col = toronto_onehot.drop('Neighborhood', axis=1)
other_col

,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1490,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1491,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1492,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1493,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [213]:
fixed_columns = ['Neighborhood'] + list(other_col.columns)
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [214]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.043478,0.0,0.0,0.000000,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.045455
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,Central Bay Street,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.016393,0.0,0.0,0.016393,0.0,0.016393
4,Christie,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000


In [215]:
toronto_grouped.shape

(39, 215)

### print each neighborhood along with the top 5 most common venues

In [216]:
number_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venues', 'freq']
    temp = temp.iloc[1:,]
    temp['freq'] = temp['freq'].astype(float).round(2)
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(number_top_venues))
    print('\n')

               venues  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.09
2      Sandwich Place  0.07
3              Bakery  0.07
4  Seafood Restaurant  0.04


           venues  freq
0  Breakfast Spot  0.09
1     Coffee Shop  0.09
2  Sandwich Place  0.09
3            Café  0.09
4     Yoga Studio  0.05


             venues  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Terminal  0.11
3     Boat or Ferry  0.06
4               Bar  0.06


               venues  freq
0         Coffee Shop  0.16
1      Sandwich Place  0.08
2    Sushi Restaurant  0.07
3  Italian Restaurant  0.05
4                Café  0.05


               venues  freq
0       Grocery Store  0.25
1                Café  0.19
2                Park  0.12
3  Italian Restaurant  0.06
4          Baby Store  0.06


                 venues  freq
0      Sushi Restaurant  0.09
1   Japanese Restaurant  0.08
2           Coffee Shop  0.05
3            Restaurant  0.05
4  Fast Food Restaurant  0.03


    

In [217]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [218]:
neighborhood_venues_new = pd.DataFrame(columns=columns)
neighborhood_venues_new['Neighborhood'] = toronto_grouped['Neighborhood']
neighborhood_venues_new.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Brockton, Parkdale Village, Exhibition Place",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"CN Tower, King and Spadina, Railway Lands, Har...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Central Bay Street,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Christie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [219]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    
    row_categories_sorted = row_categories.sort_values(ascending=False)
#     print(row_categories_sorted.index.values[0:10])
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [220]:
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhood_venues_new.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhood_venues_new.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cocktail Bar,Coffee Shop,Bakery,Sandwich Place,Farmers Market,Seafood Restaurant,Beer Bar,Vegetarian / Vegan Restaurant,Cheese Shop,Indian Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Sandwich Place,Breakfast Spot,Café,Coffee Shop,Grocery Store,Restaurant,Nightclub,Japanese Restaurant,Italian Restaurant,Intersection
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Sculpture Garden,Boat or Ferry,Rental Car Location,Bar,Boutique
3,Central Bay Street,Coffee Shop,Sandwich Place,Sushi Restaurant,Café,Italian Restaurant,Japanese Restaurant,Salad Place,Restaurant,Bank,Burger Joint
4,Christie,Grocery Store,Café,Park,Italian Restaurant,Coffee Shop,Baby Store,Nightclub,Athletics & Sports,Baseball Field,Restaurant


### Cluster Neighborhoods

In [221]:
from sklearn.cluster import KMeans

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 2,
       1, 1, 1, 1, 0, 3, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1])

In [222]:
neighborhood_venues_new.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cocktail Bar,Coffee Shop,Bakery,Sandwich Place,Farmers Market,Seafood Restaurant,Beer Bar,Vegetarian / Vegan Restaurant,Cheese Shop,Indian Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Sandwich Place,Breakfast Spot,Café,Coffee Shop,Grocery Store,Restaurant,Nightclub,Japanese Restaurant,Italian Restaurant,Intersection
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Sculpture Garden,Boat or Ferry,Rental Car Location,Bar,Boutique
3,Central Bay Street,Coffee Shop,Sandwich Place,Sushi Restaurant,Café,Italian Restaurant,Japanese Restaurant,Salad Place,Restaurant,Bank,Burger Joint
4,Christie,Grocery Store,Café,Park,Italian Restaurant,Coffee Shop,Baby Store,Nightclub,Athletics & Sports,Baseball Field,Restaurant


In [223]:
neighborhood_venues_new.insert(0, 'Cluster Labels', kmeans.labels_)

In [224]:
neighborhood_venues_new.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Berczy Park,Cocktail Bar,Coffee Shop,Bakery,Sandwich Place,Farmers Market,Seafood Restaurant,Beer Bar,Vegetarian / Vegan Restaurant,Cheese Shop,Indian Restaurant
1,1,"Brockton, Parkdale Village, Exhibition Place",Sandwich Place,Breakfast Spot,Café,Coffee Shop,Grocery Store,Restaurant,Nightclub,Japanese Restaurant,Italian Restaurant,Intersection
2,1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Sculpture Garden,Boat or Ferry,Rental Car Location,Bar,Boutique
3,1,Central Bay Street,Coffee Shop,Sandwich Place,Sushi Restaurant,Café,Italian Restaurant,Japanese Restaurant,Salad Place,Restaurant,Bank,Burger Joint
4,1,Christie,Grocery Store,Café,Park,Italian Restaurant,Coffee Shop,Baby Store,Nightclub,Athletics & Sports,Baseball Field,Restaurant


In [225]:
toronto_df_filtered.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [226]:
toronto_merged = pd.merge(toronto_df_filtered, neighborhood_venues_new, how='inner', left_on=['Neighborhood'], right_on=['Neighborhood'])
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Restaurant,Pub,Café,Distribution Center,Breakfast Spot,Dessert Shop,Sandwich Place
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Sandwich Place,Clothing Store,Café,Japanese Restaurant,Hotel,Pizza Place,Bank,Middle Eastern Restaurant,Cosmetics Shop
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Italian Restaurant,Café,Cocktail Bar,Clothing Store,Restaurant,Cosmetics Shop,Japanese Restaurant,Theater,Gym
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Health Food Store,Pub,Construction & Landscaping,Yoga Studio,Cuban Restaurant,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Cocktail Bar,Coffee Shop,Bakery,Sandwich Place,Farmers Market,Seafood Restaurant,Beer Bar,Vegetarian / Vegan Restaurant,Cheese Shop,Indian Restaurant


In [227]:
toronto_merged.shape

(39, 16)

In [228]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1 Details

In [229]:
cluster1_df = toronto_merged[toronto_merged['Cluster Labels'] == 0].drop(['Borough', 'PostalCode', 'Latitude', 'Longitude', 'Cluster Labels'], axis=1)
cluster1_df

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,The Danforth East,Coffee Shop,Park,Convenience Store,Cuban Restaurant,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
33,Rosedale,Park,Playground,Trail,Yoga Studio,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


## Cluster 2 Details

In [230]:
cluster2_df = toronto_merged[toronto_merged['Cluster Labels'] == 1].drop(['Borough', 'PostalCode', 'Latitude', 'Longitude', 'Cluster Labels'], axis=1)
cluster2_df

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Bakery,Park,Restaurant,Pub,Café,Distribution Center,Breakfast Spot,Dessert Shop,Sandwich Place
1,"Garden District, Ryerson",Coffee Shop,Sandwich Place,Clothing Store,Café,Japanese Restaurant,Hotel,Pizza Place,Bank,Middle Eastern Restaurant,Cosmetics Shop
2,St. James Town,Coffee Shop,Italian Restaurant,Café,Cocktail Bar,Clothing Store,Restaurant,Cosmetics Shop,Japanese Restaurant,Theater,Gym
3,The Beaches,Health Food Store,Pub,Construction & Landscaping,Yoga Studio,Cuban Restaurant,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
4,Berczy Park,Cocktail Bar,Coffee Shop,Bakery,Sandwich Place,Farmers Market,Seafood Restaurant,Beer Bar,Vegetarian / Vegan Restaurant,Cheese Shop,Indian Restaurant
5,Central Bay Street,Coffee Shop,Sandwich Place,Sushi Restaurant,Café,Italian Restaurant,Japanese Restaurant,Salad Place,Restaurant,Bank,Burger Joint
6,Christie,Grocery Store,Café,Park,Italian Restaurant,Coffee Shop,Baby Store,Nightclub,Athletics & Sports,Baseball Field,Restaurant
7,"Richmond, Adelaide, King",Coffee Shop,Café,Sandwich Place,Gym,Clothing Store,Sushi Restaurant,Restaurant,Steakhouse,Japanese Restaurant,Cosmetics Shop
8,"Dufferin, Dovercourt Village",Gym,Middle Eastern Restaurant,Brewery,Bar,Bank,Bakery,Supermarket,Café,Liquor Store,Grocery Store
10,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Café,Hotel,Aquarium,Pizza Place,Scenic Lookout,Restaurant,Sandwich Place,Park,Brewery


## Cluster 3 Details

In [236]:
cluster3_df = toronto_merged[toronto_merged['Cluster Labels'] == 2].drop(['Borough', 'PostalCode', 'Latitude', 'Longitude', 'Cluster Labels'], axis=1)
cluster3_df

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"Moore Park, Summerhill East",Tennis Court,Creperie,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega


## Cluster 4 Details

In [237]:
cluster4_df = toronto_merged[toronto_merged['Cluster Labels'] == 3].drop(['Borough', 'PostalCode', 'Latitude', 'Longitude', 'Cluster Labels'], axis=1)
cluster4_df

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,Fast Food Restaurant,Garden,Music Venue,Yoga Studio,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


## Cluster 5 Details

In [238]:
cluster5_df = toronto_merged[toronto_merged['Cluster Labels'] == 4].drop(['Borough', 'PostalCode', 'Latitude', 'Longitude', 'Cluster Labels'], axis=1)
cluster5_df

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,Park,Photography Studio,Bus Line,Swim School,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
